# Python Assignment: Building and Testing a REST API with FastAPI and Swagger Documentation

This assignment will challenge you to build a robust REST API using FastAPI. You'll design data models, implement CRUD (Create, Read, Update, Delete) operations, handle validation and errors, and leverage FastAPI's automatic Swagger (OpenAPI) documentation. While you cannot run the FastAPI server *directly* within this notebook, this notebook will provide all the necessary code, instructions, and challenges to build, run, and test your API externally.

## **Important Setup Instructions (READ FIRST!):**

1.  **Prerequisites:** Ensure you have Python 3.7+ installed.
2.  **Virtual Environment (Recommended):**
    ```bash
    python -m venv venv
    # On Windows:
    venv\Scripts\activate
    # On macOS/Linux:
    source venv/bin/activate
    ```
3.  **Install FastAPI and Uvicorn:**
    ```bash
    pip install fastapi uvicorn[standard] pydantic
    ```
4.  **Creating Python Files:** As you progress through the assignment, you will be instructed to copy code snippets from this notebook into separate `.py` files (e.g., `main.py`, `models.py`). You will run these files from your terminal.
5.  **Running the FastAPI Application:** Once you've created your `main.py` (or similar entry point), you will run your API from your terminal using:
    ```bash
    uvicorn main:app --reload
    ```
    (Replace `main:app` with `your_module_name:your_fastapi_app_instance_name` if different).
    The `--reload` flag is great for development as it restarts the server on code changes.

6.  **Accessing Swagger UI:** Once your server is running, open your web browser and go to `http://127.0.0.1:8000/docs` to see the interactive API documentation (Swagger UI). This is where you will primarily test your endpoints.

7.  **Testing with `curl` or Postman:** You can also test your endpoints using `curl` from your terminal or a tool like Postman/Insomnia for more complex requests.


## Scenario: Simple Bookstore API

You will build a REST API to manage a collection of books. Each book will have an ID, title, author, publication year, and price.

## Part 1: Data Models and Initial Setup (30 points)

Define your data structures using Pydantic for validation and serialization.

In [ ]:
### 1.1 `models.py` - Book Data Model
Create a new file named `models.py` and add the following Pydantic models. You will need to copy this into a separate `.py` file.

```python
from pydantic import BaseModel, Field, PositiveInt, NonNegativeFloat
from typing import Optional, List
from datetime import datetime

class BookBase(BaseModel):
    title: str = Field(..., min_length=3, max_length=100, example="The Great Gatsby")
    author: str = Field(..., min_length=3, max_length=100, example="F. Scott Fitzgerald")
    publication_year: PositiveInt = Field(..., ge=1800, le=datetime.now().year, description="Year of publication")
    price: NonNegativeFloat = Field(..., gt=0.0, example=12.99)
    # Challenge: Add a new field 'genre' (string, optional, with default)
    # Challenge: Add a new field 'isbn' (string, unique, with regex validation - 10 or 13 digits, optional for now)
    # Hint: Use Field(..., regex="^(\\d{9}[\dX])|(\\d{13})$", description="ISBN-10 or ISBN-13")

class BookCreate(BookBase):
    # This model is used for creating new books. Inherits from BookBase.
    pass

class BookUpdate(BookBase):
    # This model is used for updating existing books. All fields should be optional.
    title: Optional[str] = None
    author: Optional[str] = None
    publication_year: Optional[PositiveInt] = None
    price: Optional[NonNegativeFloat] = None
    # Challenge: Make your new fields optional here too.

class BookInDB(BookBase):
    # This model represents a book as stored in our 'database' (in-memory for now).
    # It should include the 'id' which is generated by the server.
    id: str = Field(..., example="b123e456-e89b-12d3-a456-426614174000", description="Unique Book ID")

    class Config:
        from_attributes = True # Pydantic v2 equivalent for orm_mode = True in v1

# Challenge: Create a simple 'ErrorDetail' model for custom error responses (e.g., status_code, message, detail)
```

### 1.2 `main.py` - FastAPI App Setup
Create a new file named `main.py` and add the basic FastAPI application setup.

```python
from fastapi import FastAPI, HTTPException, status, Depends, Query, Path
from typing import Dict, List, Optional
from uuid import uuid4

from models import BookCreate, BookInDB, BookUpdate # Assuming you saved models.py
# from models import ErrorDetail # If you implemented the challenge

app = FastAPI(
    title="My Tough Bookstore API",
    description="A challenging API for managing a collection of books.",
    version="1.0.0",
    contact={
        "name": "Your Name",
        "email": "your.email@example.com",
    },
    license_info={
        "name": "MIT License",
        "url": "[https://opensource.org/licenses/MIT](https://opensource.org/licenses/MIT)",
    }
)

# In-memory database (for simplicity, real apps use actual databases)
books_db: Dict[str, BookInDB] = {}

# Initial dummy data (optional but good for testing)
initial_books = [
    {"title": "1984", "author": "George Orwell", "publication_year": 1949, "price": 9.99},
    {"title": "Brave New World", "author": "Aldous Huxley", "publication_year": 1932, "price": 10.50},
    {"title": "The Lord of the Rings", "author": "J.R.R. Tolkien", "publication_year": 1954, "price": 25.00}
]

for book_data in initial_books:
    book_id = str(uuid4())
    books_db[book_id] = BookInDB(id=book_id, **book_data)

@app.get("/", tags=["Root"], summary="Root endpoint of the API")
async def read_root():
    return {"message": "Welcome to the Tough Bookstore API!"}

# After adding the above, save main.py and run: uvicorn main:app --reload
# Then, open [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs) in your browser.
```

## Part 2: CRUD Endpoints and Path/Query Parameters (40 points)

Implement the core API functionalities for managing books.

In [ ]:
### 2.1 Create Book (`POST /books/`)
-   **Description:** Creates a new book entry.
-   **Request Body:** `BookCreate` model.
-   **Response:** `BookInDB` model with the generated `id`.
-   **Status Code:** `201 Created`.
-   **Challenge:** Add a custom `detail` message to the `HTTPException` if a book with the same `isbn` (if you implemented it) already exists. Use `status.HTTP_409_CONFLICT`.

```python
# Add this to main.py

@app.post(
    "/books/",
    response_model=BookInDB,
    status_code=status.HTTP_201_CREATED,
    tags=["Books"],
    summary="Create a new book",
    description="Adds a new book to the inventory with a unique generated ID."
)
async def create_book(book: BookCreate):
    book_id = str(uuid4())
    new_book = BookInDB(id=book_id, **book.model_dump()) # Use model_dump() for Pydantic v2
    # Challenge: Check for existing ISBN and raise 409 conflict
    books_db[book_id] = new_book
    return new_book
```

In [ ]:
### 2.2 Get All Books (`GET /books/`)
-   **Description:** Retrieves a list of all books.
-   **Response:** `List[BookInDB]`.
-   **Challenge:** Implement **pagination**. Allow `skip` (int, default 0) and `limit` (int, default 10, max 100) as query parameters.
-   **Challenge:** Implement **filtering**. Allow `author` (string, optional) and `publication_year` (int, optional) as query parameters to filter results.

```python
# Add this to main.py

@app.get(
    "/books/",
    response_model=List[BookInDB],
    tags=["Books"],
    summary="Retrieve all books",
    description="Retrieves a list of all books, with optional filtering and pagination."
)
async def get_all_books(
    skip: int = Query(0, ge=0, description="Number of items to skip"),
    limit: int = Query(10, ge=1, le=100, description="Maximum number of items to return"),
    author: Optional[str] = Query(None, description="Filter by author name"),
    publication_year: Optional[int] = Query(None, ge=1800, description="Filter by publication year")
):
    filtered_books = list(books_db.values())

    if author:
        filtered_books = [book for book in filtered_books if author.lower() in book.author.lower()]
    if publication_year:
        filtered_books = [book for book in filtered_books if book.publication_year == publication_year]

    # Apply pagination AFTER filtering
    return filtered_books[skip : skip + limit]
```

In [ ]:
### 2.3 Get Book by ID (`GET /books/{book_id}`)
-   **Description:** Retrieves a single book by its unique ID.
-   **Path Parameter:** `book_id` (string).
-   **Response:** `BookInDB`.
-   **Error Handling:** Return `404 Not Found` if the `book_id` does not exist.
-   **Challenge:** Add a custom response model for the 404 error using FastAPI's `responses` parameter in the decorator, and link it to your `ErrorDetail` model (if implemented).

```python
# Add this to main.py

@app.get(
    "/books/{book_id}",
    response_model=BookInDB,
    tags=["Books"],
    summary="Retrieve a single book by ID",
    responses={
        status.HTTP_404_NOT_FOUND: {"description": "Book not found", "model": dict} # Replace dict with ErrorDetail if implemented
    }
)
async def get_book_by_id(book_id: str = Path(..., example="b123e456-e89b-12d3-a456-426614174000", description="The ID of the book to retrieve")):
    book = books_db.get(book_id)
    if not book:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Book not found."
        )
    return book
```

In [ ]:
### 2.4 Update Book (`PUT /books/{book_id}`)
-   **Description:** Updates an existing book by its ID. It should replace all fields.
-   **Path Parameter:** `book_id`.
-   **Request Body:** `BookCreate` (because it's a full replacement).
-   **Response:** `BookInDB`.
-   **Error Handling:** `404 Not Found` if `book_id` doesn't exist.

```python
# Add this to main.py

@app.put(
    "/books/{book_id}",
    response_model=BookInDB,
    tags=["Books"],
    summary="Update an existing book",
    description="Replaces all fields of an existing book with the provided data."
)
async def update_book(
    book_id: str = Path(..., description="The ID of the book to update"),
    book_data: BookCreate
):
    if book_id not in books_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Book not found."
        )
    updated_book = BookInDB(id=book_id, **book_data.model_dump())
    books_db[book_id] = updated_book
    return updated_book
```

In [ ]:
### 2.5 Partially Update Book (`PATCH /books/{book_id}`)
-   **Description:** Partially updates an existing book by its ID. Only provided fields are updated.
-   **Path Parameter:** `book_id`.
-   **Request Body:** `BookUpdate`.
-   **Response:** `BookInDB`.
-   **Error Handling:** `404 Not Found` if `book_id` doesn't exist.

```python
# Add this to main.py

@app.patch(
    "/books/{book_id}",
    response_model=BookInDB,
    tags=["Books"],
    summary="Partially update an existing book",
    description="Updates only the provided fields of an existing book."
)
async def patch_book(
    book_id: str = Path(..., description="The ID of the book to partially update"),
    book_data: BookUpdate
):
    existing_book = books_db.get(book_id)
    if not existing_book:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Book not found."
        )

    update_data = book_data.model_dump(exclude_unset=True) # Exclude fields not provided in request
    updated_item = existing_book.model_copy(update=update_data) # Pydantic v2 way to update

    books_db[book_id] = updated_item
    return updated_item
```

In [ ]:
### 2.6 Delete Book (`DELETE /books/{book_id}`)
-   **Description:** Deletes a book by its ID.
-   **Path Parameter:** `book_id`.
-   **Response:** A simple success message.
-   **Status Code:** `204 No Content` on successful deletion.
-   **Error Handling:** `404 Not Found` if `book_id` doesn't exist.

```python
# Add this to main.py

@app.delete(
    "/books/{book_id}",
    status_code=status.HTTP_204_NO_CONTENT,
    tags=["Books"],
    summary="Delete a book",
    description="Removes a book from the inventory by its ID."
)
async def delete_book(book_id: str = Path(..., description="The ID of the book to delete")):
    if book_id not in books_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Book not found."
        )
    del books_db[book_id]
    return # No content for 204
```

## Part 3: Advanced Features and Documentation (30 points)

Enhance your API with more advanced features and ensure comprehensive documentation.

In [ ]:
### 3.1 Custom Error Handling (If not done in Part 2)
-   Implement a custom exception handler for a specific error type (e.g., `ValueError` raised by your `Book` model validation if you added it, or a custom `BookAlreadyExistsError`).
-   The handler should return a structured JSON response (ideally using your `ErrorDetail` model).

```python
# Add this to main.py

# from fastapi.responses import JSONResponse
# from fastapi.exceptions import RequestValidationError

# @app.exception_handler(RequestValidationError)
# async def validation_exception_handler(request: Request, exc: RequestValidationError):
#     return JSONResponse(
#         status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
#         content=jsonable_encoder({"detail": exc.errors(), "body": exc.body}),
#     )

# Challenge: Implement a custom exception for ISBN conflict if you haven't already.
# class DuplicateISBNError(Exception):
#    def __init__(self, isbn: str):
#        self.isbn = isbn

# @app.exception_handler(DuplicateISBNError)
# async def duplicate_isbn_exception_handler(request: Request, exc: DuplicateISBNError):
#     return JSONResponse(
#         status_code=status.HTTP_409_CONFLICT,
#         content={"message": f"Book with ISBN {exc.isbn} already exists.", "code": "DUPLICATE_ISBN"}
#     )
```

In [ ]:
### 3.2 Dependencies / Dependency Injection
-   Create a simple dependency function that could represent a 'database session' or 'authentication check'.
-   Apply this dependency to at least one of your API endpoints (e.g., `create_book` or `get_all_books`).

```python
# Add this to main.py

async def get_db_session():
    # In a real app, this would yield a database session
    print("Getting database session...")
    try:
        yield "fake_db_session_123" # This simulates a dependency that yields resources
    finally:
        print("Closing database session.")

# Example usage on an endpoint:
# @app.get("/books/", response_model=List[BookInDB], tags=["Books"])
# async def get_all_books(
#     db_session: str = Depends(get_db_session),
#     skip: int = Query(0, ge=0), limit: int = Query(10, ge=1, le=100),
#     author: Optional[str] = None, publication_year: Optional[int] = None
# ):
#     print(f"Using DB Session: {db_session}")
#     # ... rest of your get_all_books logic
#     pass

```

In [ ]:
### 3.3 Response Model Customization
-   For the `GET /books/{book_id}` endpoint, modify its `response_model` to return only a subset of fields (e.g., `title`, `author`, `price`) in a new Pydantic model called `BookSummary`.
-   Ensure the Swagger UI reflects this change correctly.

```python
# Add to models.py

class BookSummary(BaseModel):
    title: str
    author: str
    price: NonNegativeFloat

# Then, in main.py, modify the decorator for get_book_by_id:
# @app.get(
#     "/books/{book_id}",
#     response_model=BookSummary, # Change here!
#     tags=["Books"],
#     summary="Retrieve a single book by ID",
#     ...
# )

In [ ]:
### 3.4 Request Body Examples and Descriptions
-   For `POST /books/` and `PUT /books/{book_id}`, add `examples` to your `Field` definitions in `BookBase` (or `BookCreate`/`BookUpdate`) to provide clearer Swagger documentation.
-   Add more detailed `description` arguments to your `Field` and `Query`/`Path` parameters for better readability in Swagger UI.

*(This was partially covered in Part 1 and 2 code snippets, ensure you've expanded on it.)*


## Part 4: Testing and Reflection (Bonus - 10 points)

After implementing all endpoints and running your API, reflect on your work and how to test it.

In [ ]:
### 4.1 Manual Testing via Swagger UI and `curl`
Provide `curl` commands (or detailed Postman/Insomnia instructions) to test each of your CRUD endpoints, including:

1.  **Create Book:** (POST request with JSON body)
2.  **Get All Books:** (GET request, test with and without pagination/filters)
3.  **Get Book by ID:** (GET request, test valid and invalid IDs)
4.  **Update Book:** (PUT request, full replacement)
5.  **Partially Update Book:** (PATCH request, partial update)
6.  **Delete Book:** (DELETE request)

```bash
# Example curl for GET /books/
# curl -X GET "http://127.0.0.1:8000/books/?skip=0&limit=5&author=George%20Orwell"

# Your curl commands here:

# 1. Create Book:
#

# 2. Get All Books (with filters/pagination):
#

# 3. Get Book by ID:
#

# 4. Update Book:
#

# 5. Partially Update Book:
#

# 6. Delete Book:
#
```

In [ ]:
### 4.2 Automated Testing (Conceptual)
How would you approach writing automated unit/integration tests for your FastAPI application? Mention specific Python libraries you would use (e.g., `pytest`, `httpx` or `TestClient` from `fastapi.testclient`) and give examples of scenarios you would test for a `create_book` endpoint.

```python
# Your conceptual testing strategy here (as comments or multi-line string)

from fastapi.testclient import TestClient
# from main import app # Assuming your FastAPI app instance is named 'app' in main.py

# client = TestClient(app)

# def test_create_book():
#     # ... test logic here
#     pass
```

### 4.3 Scalability and Persistence (Conceptual)
Your current 'database' is in-memory. Briefly explain how you would transition this API to use a real database (e.g., PostgreSQL, SQLite) using an ORM like SQLAlchemy with FastAPI. What changes would be needed in your `models.py` and `main.py`?

```python
# Your scalability/persistence ideas here (as comments or multi-line string)
```